In [1]:
%pip install  dgl -f https://data.dgl.ai/wheels/torch-2.3/repo.html
%pip install labml-nn

Looking in links: https://data.dgl.ai/wheels/torch-2.3/repo.html
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pydantic


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import dgl.sparse as dglsp
import torch
import torch.nn as nn
import torch.nn.functional as F


class LinearNeuralNetwork(nn.Module):
    def __init__(self, nfeat, nclass, bias=True):
        super(LinearNeuralNetwork, self).__init__()
        self.W = nn.Linear(nfeat, nclass, bias=bias)

    def forward(self, x):
        return self.W(x)


def symmetric_normalize_adjacency(graph):
    """Symmetric normalize graph adjacency matrix."""
    indices = torch.stack(graph.edges())
    n = graph.num_nodes()
    adj = dglsp.spmatrix(indices, shape=(n, n))
    deg_invsqrt = dglsp.diag(adj.sum(0)) ** -0.5
    return deg_invsqrt @ adj @ deg_invsqrt


def model_test(model, embeds):
    model.eval()
    with torch.no_grad():
        output = model(embeds)
        pred = output.argmax(dim=-1)
        test_mask, tv_mask = model.test_mask, model.tv_mask
        loss_tv = F.mse_loss(output[tv_mask], model.label_one_hot[tv_mask])
    accs = []
    for mask in [tv_mask, test_mask]:
        accs.append(
            float((pred[mask] == model.label[mask]).sum() / mask.sum()))
    return loss_tv.item(), accs[0], accs[1], pred

c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
c:\Users\Taneem\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import dgl.sparse as dglsp
import torch.nn as nn
import torch.nn.functional as F


class OGC(nn.Module):
    def __init__(self, graph):
        super(OGC, self).__init__()
        self.linear_clf = LinearNeuralNetwork(
            nfeat=graph.ndata["feat"].shape[1],
            nclass=graph.ndata["label"].max().item() + 1,
            bias=False,
        )

        self.label = graph.ndata["label"]
        self.label_one_hot = F.one_hot(graph.ndata["label"]).float()
        # LIM trick, else use both train and val set to construct this matrix.
        self.label_idx_mat = dglsp.diag(graph.ndata["train_mask"]).float()

        self.test_mask = graph.ndata["test_mask"]
        self.tv_mask = graph.ndata["train_mask"] + graph.ndata["val_mask"]

    def forward(self, x):
        return self.linear_clf(x)

    def update_embeds(self, embeds, lazy_adj, args):
        """Update classifier's weight by training a linear supervised model."""
        pred_label = self(embeds).data
        clf_weight = self.linear_clf.W.weight.data

        # Update the smoothness loss via LGC.
        embeds = dglsp.spmm(lazy_adj, embeds)

        # Update the supervised loss via SEB.
        deriv_sup = 2 * dglsp.matmul(
            dglsp.spmm(self.label_idx_mat, -self.label_one_hot + pred_label),
            clf_weight,
        )
        embeds = embeds - args.lr_sup * deriv_sup

        args.lr_sup = args.lr_sup * args.decline
        return embeds

In [5]:
import argparse
import time

import dgl.sparse as dglsp

import torch.nn.functional as F
import torch.optim as optim
from dgl import AddSelfLoop
from dgl.data import CoraGraphDataset

parser = argparse.ArgumentParser()
args = parser.parse_args([])
args.dataset = "cora"
args.decline = 0.9
args.lr_sup = 0.001
args.lr_clf = 0.5
args.beta = 0.1
args.max_sim_rate = 0.995
args.max_patience = 2
args.device = torch.device("cpu")

In [6]:
def train(model, embeds, lazy_adj, args):
    patience = 0
    _, _, last_acc, last_output = model_test(model, embeds)

    tv_mask = model.tv_mask
    optimizer = optim.SGD(model.parameters(), lr=args.lr_clf)

    for i in range(64):
        model.train()
        output = model(embeds)
        loss_tv = F.mse_loss(
            output[tv_mask], model.label_one_hot[tv_mask], reduction="sum"
        )
        optimizer.zero_grad()
        loss_tv.backward()
        optimizer.step()

        # Updating node embeds by LGC and SEB jointly.
        embeds = model.update_embeds(embeds, lazy_adj, args)

        loss_tv, acc_tv, acc_test, pred = model_test(model, embeds)
        print(
            "epoch {} loss_tv {:.4f} acc_tv {:.4f} acc_test {:.4f}".format(
                i + 1, loss_tv, acc_tv, acc_test
            )
        )

        sim_rate = float(int((pred == last_output).sum()) / int(pred.shape[0]))
        if sim_rate > args.max_sim_rate:
            patience += 1
            if patience > args.max_patience:
                break
        last_acc = acc_test
        last_output = pred
    return last_acc

In [7]:
!nvidia-smi

Tue Dec 31 19:02:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P3              11W /  30W |    147MiB /  4096MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
transform = AddSelfLoop()
data = CoraGraphDataset(transform=transform)
graph = data[0].to(args.device)

C:\Users\Taneem\.dgl\cora_v2.zip: 100%|██████████| 132k/132k [00:02<00:00, 57.3kB/s] 


Extracting file to C:\Users\Taneem\.dgl\cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [9]:
graph.has_edges_between([0], [1])

tensor([False])

In [10]:
def pagerank(net, weights={}, q=0.5, eps=0.01, maxIters=500, verbose=False, weightName='weight'):

    incomingTeleProb = {}
    prevVisitProb = {}
    currVisitProb = {}
    N = net.number_of_nodes()

    totWeight = sum([w for v, w in weights.items()])

    if totWeight == 0:
        incomingTeleProb = dict.fromkeys(net, 1.0 / N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()
    else:
        minPosWeight = 1.0
        for v, weight in weights.items():
            if weight == 0:
                continue
            minPosWeight = min(minPosWeight, 1.0 * weight / totWeight)

        smallWeight = minPosWeight / (10 ** 6)

        for v in net.nodes():
            weight = weights.get(v, 0.0)
            incomingTeleProb[v] = 1.0 * \
                (weight + smallWeight) / (totWeight + smallWeight * N)
        prevVisitProb = incomingTeleProb.copy()
        currVisitProb = incomingTeleProb.copy()

    outDeg = {}
    zeroDegNodes = set()
    for v in net.nodes():
        outDeg[v] = 1.0 * net.out_degree(v, weight=weightName)
        if outDeg[v] == 0:
            zeroDegNodes.add(v)

    iters = 0
    finished = False
    while not finished:
        iters += 1
        prevVisitProb = currVisitProb.copy()
        maxDiff = 0

        zSum = sum([prevVisitProb[x] for x in zeroDegNodes]) / N

        for v in net.nodes():
            eSum = 0
            for u in net.predecessors(v):
                # Handle missing 'weight' attribute by assuming it's 1.0
                w_uv = 1.0 * net[u][v].get(weightName, 1.0)
                eSum += w_uv / outDeg[u] * prevVisitProb[u]

            currVisitProb[v] = q * incomingTeleProb[v] + \
                (1 - q) * (eSum + zSum)
            maxDiff = max(maxDiff, abs(
                (prevVisitProb[v] - currVisitProb[v]) / currVisitProb[v]))

        if verbose:
            print('\tIteration %d, max difference %f' % (iters, maxDiff))
            if maxDiff < eps:
                print('PageRank converged after %d iterations, max difference %f.' % (
                    iters, maxDiff))

        if iters >= maxIters:
            print(
                'WARNING: PageRank terminated because max iters (%d) was reached.' % (maxIters))

        finished = (maxDiff < eps) or (iters >= maxIters)

    return currVisitProb

In [11]:
import networkx as nx

# Convert DGL graph to NetworkX
nx_graph = graph.to_networkx()

# Run PageRank
pagerank_scores = pagerank(nx_graph)

# Assuming graph.ndata['feat'] or similar contains node features, map DGL nodes to NetworkX nodes
# If the DGL graph uses integer indices for nodes, you can map directly

# Print pagerank scores (optional)
print(pagerank_scores)

{0: 0.00036052501717742354, 1: 0.0004028661685851042, 2: 0.00042950848679268656, 3: 0.00036927621861152144, 4: 0.00035722113227557104, 5: 0.0003712386186454783, 6: 0.0003268850780867359, 7: 0.00036927621861152144, 8: 0.0003311581532613355, 9: 0.00033604025850795356, 10: 0.00036371323256205717, 11: 0.000288389696149014, 12: 0.00043213180891019446, 13: 0.0002743402678377291, 14: 0.00034921938544734196, 15: 0.00032836603464356077, 16: 0.0003538052070250879, 17: 0.0004068781812034786, 18: 0.00040390072061154304, 19: 0.00032241995629062254, 20: 0.0004061654847465194, 21: 0.0003539112306090031, 22: 0.00034467036618850356, 23: 0.0003096395294460056, 24: 0.00041858940265620875, 25: 0.00036733492195301423, 26: 0.0003999421957619074, 27: 0.0003681953744845908, 28: 0.00029932130794493427, 29: 0.0002887645750047761, 30: 0.00041450899066961065, 31: 0.00036927621861152144, 32: 0.00033383405751729, 33: 0.0005187340768002362, 34: 0.00026996422662422273, 35: 0.0003224533895005945, 36: 0.000476829380820

In [12]:
import torch as th

features = graph.ndata["feat"]
adj = symmetric_normalize_adjacency(graph)
# I_N = dglsp.identity((features.shape[0], features.shape[0]))
# print(I_N)

# Lazy random walk (also known as lazy graph convolution).
# lazy_adj = dglsp.add((1 - args.beta) * I_N, args.beta * adj).to(args.device)
# print(lazy_adj)

# Convert PageRank scores to a tensor, matching DGL graph node indices with NetworkX nodes
pagerank_tensor = th.tensor(
    [pagerank_scores[int(node.item())] for node in graph.nodes()])

# Normalize PageRank scores (optional)
pagerank_tensor /= pagerank_tensor.sum()

print(pagerank_tensor)

# Assuming pagerank_tensor has been defined earlier and is on the correct device
# Create a sparse diagonal matrix from the PageRank scores
num_nodes = pagerank_tensor.shape[0]  # Get the number of nodes
pagerank_diag = dglsp.diag(pagerank_tensor).to(
    args.device)  # Convert to a DGL SparseMatrix

# Now, modify lazy_adj by incorporating PageRank scores
# Ensure adj is a DGL SparseMatrix as well
lazy_adj = dglsp.add((1 - args.beta) * pagerank_diag, args.beta * adj)

print(lazy_adj)  # Check the lazy adjacency matrix

tensor([0.0004, 0.0004, 0.0004,  ..., 0.0004, 0.0004, 0.0003])
SparseMatrix(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                             [   0,  633, 1862,  ..., 1473, 2706, 2707]]),
             values=tensor([0.0253, 0.0250, 0.0224,  ..., 0.0200, 0.0200, 0.0203]),
             shape=(2708, 2708), nnz=13264)


In [13]:
model = OGC(graph).to(args.device)

In [14]:
sum(p.numel() for p in model.parameters())

10031

In [15]:
start_time = time.time()
res = train(model, features, lazy_adj, args)
time_tot = time.time() - start_time

print(f"Test Acc:{res:.4f}")
print(f"Total Time:{time_tot:.4f}")

epoch 1 loss_tv 0.1344 acc_tv 0.5797 acc_test 0.4440
epoch 2 loss_tv 0.1363 acc_tv 0.5031 acc_test 0.5280
epoch 3 loss_tv 0.1390 acc_tv 0.6313 acc_test 0.4690
epoch 4 loss_tv 0.1386 acc_tv 0.6031 acc_test 0.5540
epoch 5 loss_tv 0.1390 acc_tv 0.7156 acc_test 0.6310
epoch 6 loss_tv 0.1391 acc_tv 0.7391 acc_test 0.6690
epoch 7 loss_tv 0.1393 acc_tv 0.7797 acc_test 0.7110
epoch 8 loss_tv 0.1395 acc_tv 0.7844 acc_test 0.7330
epoch 9 loss_tv 0.1397 acc_tv 0.7937 acc_test 0.7370
epoch 10 loss_tv 0.1399 acc_tv 0.7969 acc_test 0.7450
epoch 11 loss_tv 0.1401 acc_tv 0.8016 acc_test 0.7450
epoch 12 loss_tv 0.1403 acc_tv 0.8016 acc_test 0.7440
epoch 13 loss_tv 0.1404 acc_tv 0.8016 acc_test 0.7400
epoch 14 loss_tv 0.1406 acc_tv 0.8031 acc_test 0.7380
Test Acc:0.7400
Total Time:6.0121
